In [1]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
sid = SentimentIntensityAnalyzer()

In [10]:
df = pd.read_json('amazon_evandro.json')
df.replace(r'\n', '', regex=True, inplace=True)
df.replace(r'\xa0', '', regex=True, inplace=True)
df.replace(r'\r', '', regex=True, inplace=True)
df.head()

ValueError: Trailing data

In [5]:
df.isnull().sum()

Title          0
Date           0
Contributor    0
Content        0
Link           0
Role           0
dtype: int64

In [6]:
df.dropna(inplace=True)
df.drop('Role', axis=1, inplace=True)

In [7]:
sid.polarity_scores(df.iloc[0]['Content'])

{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'compound': 0.9955}

In [8]:
df.Content[0]

'Partnership will expand FPT Software’s client offerings with Amelia’s market-leading AI solutions.NEW YORK, NY, October 20, 2021 — Amelia, a leading Enterprise AI software company, today announced its new partnership with FPT Software, Vietnam’s largest technology services provider. FPT Software provides technical consulting, streamlining and cost-savings services to thousands of global customers, including Fortune Global 500 companies.FPT Software will include Amelia’s industry-leading AI solutions within its client offerings. Through this partnership, the provider will enhance its services by providing clients with the most modern, capable and transformative solutions. By leveraging its “people, process and technology” approach and Amelia’s solutions, FPT Software will help its customers streamline operations and reduce costs.“We are excited to partner withAmeliaand incorporate their Conversational AI capabilities into our intelligent automation solutions along with our akaBotTM pla

In [9]:
df['Scores'] = df['Content'].apply(lambda content: sid.polarity_scores(content))
df.head()

,Title,Date,Contributor,Content,Link,Scores
0,FPT Software Partners With Amelia to Enhance C...,"October 20, 2021 • 2 minute read",,Partnership will expand FPT Software’s client ...,https://amelia.ai/press-release/fpt-software-p...,"{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'comp..."
1,Amelia Hired By Staffmark Group to Bring Conve...,"July 29, 2021 • 2 minute read",,Staffmark Group engages Amelia to deliver inte...,https://amelia.ai/press-release/amelia-hired-b...,"{'neg': 0.004, 'neu': 0.87, 'pos': 0.126, 'com..."
2,OrangeShine Hires Amelia to Automate IT Operat...,"May 28, 2021 • 2 minute read",,The leading B2B online fashion wholesale marke...,https://amelia.ai/press-release/orangeshine-hi...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp..."
3,Amelia Recognized as a Leader in Intelligent V...,"May 10, 2021 • 2 minute read",,New report on Hyper-Intelligent Automation hig...,https://amelia.ai/press-release/amelia-recogni...,"{'neg': 0.003, 'neu': 0.898, 'pos': 0.099, 'co..."
4,Kenneth S. Nugent Law Firm Forms Subcompany 1L...,"April 26, 2021 • 2 minute read",,1LAW provides Amelia as a platform that law fi...,https://amelia.ai/press-release/kenneth-s-nuge...,"{'neg': 0.015, 'neu': 0.857, 'pos': 0.128, 'co..."


In [10]:
df['Compound'] = df['Scores'].apply(lambda d: d['compound'])
df['Overall Score'] = df['Compound'].apply(lambda score: 'POS' if score >=0 else 'NEG')
df.head()

,Title,Date,Contributor,Content,Link,Scores,Compound,Overall Score
0,FPT Software Partners With Amelia to Enhance C...,"October 20, 2021 • 2 minute read",,Partnership will expand FPT Software’s client ...,https://amelia.ai/press-release/fpt-software-p...,"{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'comp...",0.9955,POS
1,Amelia Hired By Staffmark Group to Bring Conve...,"July 29, 2021 • 2 minute read",,Staffmark Group engages Amelia to deliver inte...,https://amelia.ai/press-release/amelia-hired-b...,"{'neg': 0.004, 'neu': 0.87, 'pos': 0.126, 'com...",0.9962,POS
2,OrangeShine Hires Amelia to Automate IT Operat...,"May 28, 2021 • 2 minute read",,The leading B2B online fashion wholesale marke...,https://amelia.ai/press-release/orangeshine-hi...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp...",0.9962,POS
3,Amelia Recognized as a Leader in Intelligent V...,"May 10, 2021 • 2 minute read",,New report on Hyper-Intelligent Automation hig...,https://amelia.ai/press-release/amelia-recogni...,"{'neg': 0.003, 'neu': 0.898, 'pos': 0.099, 'co...",0.9958,POS
4,Kenneth S. Nugent Law Firm Forms Subcompany 1L...,"April 26, 2021 • 2 minute read",,1LAW provides Amelia as a platform that law fi...,https://amelia.ai/press-release/kenneth-s-nuge...,"{'neg': 0.015, 'neu': 0.857, 'pos': 0.128, 'co...",0.9965,POS


In [11]:
df[df['Overall Score'] == 'NEG']

,Title,Date,Contributor,Content,Link,Scores,Compound,Overall Score


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=.9, min_df=.1, stop_words= 'english')
dtm = cv.fit_transform(df['Content'])

In [13]:
dtm

<197x379 sparse matrix of type '<class 'numpy.int64'>'
	with 20017 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=10)
LDA.fit(dtm)

LatentDirichletAllocation()

In [17]:
first_topic = LDA.components_[0]
first_topic.argsort()

array([132, 163, 148, 283, 375, 158,   9, 188, 328, 302, 137,   3,  23,
       152, 352, 343, 299, 128, 325,  44, 212,  20,  30, 156, 141, 292,
       169,  92, 254, 263,  67,  73, 233, 223, 121,  66, 246, 224, 140,
       240, 304, 170,  77, 108,  33, 314, 181, 326, 143, 348,  85, 204,
       255, 248, 344, 285, 134, 329, 282, 228, 318, 346,   6, 198, 173,
        55, 171,   8,  53, 320, 368, 356,  94,  29, 303, 256,  24,   1,
         5, 226, 172, 250, 330, 122, 157,  28, 264,  86, 324, 162, 274,
         4, 151, 307,  27, 178, 145, 327,  71, 160, 355, 378, 187, 270,
        96, 185, 290,  88,  31, 113,  47,  95,  89, 139, 261, 241,  72,
       206,  79,  74,   2, 275, 237, 280, 247, 340, 251, 203, 235,  93,
       291,  70, 176, 217, 116,  14, 372,  65, 265, 309, 259, 109,  17,
       149, 117, 230, 249, 199, 215,  49, 168, 184,  32, 272,  11,  46,
       333,  35, 153, 222, 370, 351, 135, 111, 288, 124, 194, 279,  84,
       268, 310, 107, 317, 337, 105, 112, 262, 136, 313, 189,  2

In [18]:
top_ten = first_topic.argsort()[-10:]
for index in top_ten:
    print(cv.get_feature_names_out()[index])

labor
amelia
digital
automation
business
dube
human
ipsoft
ceo
chetan


In [19]:
import sys

orig_stdout = sys.stdout
f = open(
    'df.txt', 'w'
)
sys.stdout = f


for index, topic in enumerate(LDA.components_):
    dataframe = pd.DataFrame(columns=[f'THE TOP 15 WORDS FOR TOPIC #{index}'], data=[cv.get_feature_names_out()[index] for index in topic.argsort()[-15:]])
    print(dataframe)

sys.stdout = orig_stdout
f.close()

In [20]:
df2 = pd.read_table('df.txt', header=None)
import numpy as np
arr = np.array(df2)
dataframe = pd.DataFrame(arr.reshape((10, 16))).T
headers = dataframe.iloc[0]
new_df = pd.DataFrame(data=dataframe.values[1:], columns=headers)
new_df

,THE TOP 15 WORDS FOR TOPIC #0,THE TOP 15 WORDS FOR TOPIC #1,THE TOP 15 WORDS FOR TOPIC #2,THE TOP 15 WORDS FOR TOPIC #3,THE TOP 15 WORDS FOR TOPIC #4,THE TOP 15 WORDS FOR TOPIC #5,THE TOP 15 WORDS FOR TOPIC #6,THE TOP 15 WORDS FOR TOPIC #7,THE TOP 15 WORDS FOR TOPIC #8,THE TOP 15 WORDS FOR TOPIC #9
0,0 today,0 execute,0 automation,0 cognitive,0 help,0 solutions,0 help,0 issues,0 existing,0 market
1,1 post,1 hr,1 technology,1 employees,1 experiences,1 digital,1 director,1 new,1 need,1 experiences
2,2 customer,2 people,2 products,2 value,2 solutions,2 market,2 life,2 workers,2 executives,2 business
3,3 future,3 employees,3 enterprise,3 sure,3 support,3 enterprise,3 technology,3 customers,3 said,3 digital
4,4 companies,4 looking,4 recent,4 service,4 enterprise,4 conversational,4 hr,4 work,4 global,4 report
5,5 labor,5 provider,5 read,5 workforce,5 company,5 automation,5 cognitive,5 like,5 roles,5 leading
6,6 amelia,6 work,6 voice,6 chetan,6 new,6 new,6 article,6 intelligence,6 organizations,6 operations
7,7 digital,7 helping,7 focus,7 work,7 end,7 operations,7 information,7 customer,7 skills,7 human
8,8 automation,8 possible,8 ed,8 companies,8 digital,8 human,8 new,8 tasks,8 pandemic,8 enterprise
9,9 business,9 faster,9 op,9 hybrid,9 conversational,9 technology,9 global,9 employees,9 organization,9 new


In [21]:
new_df.to_excel('amelia_topic.xlsx', index=False)

In [22]:
topic_results = LDA.transform(dtm)
topic_results[0].argmax()

4

In [23]:
df['Topic'] = topic_results.argmax(axis=1)

In [24]:
df.head()

,Title,Date,Contributor,Content,Link,Scores,Compound,Overall Score,Topic
0,FPT Software Partners With Amelia to Enhance C...,"October 20, 2021 • 2 minute read",,Partnership will expand FPT Software’s client ...,https://amelia.ai/press-release/fpt-software-p...,"{'neg': 0.0, 'neu': 0.869, 'pos': 0.131, 'comp...",0.9955,POS,4
1,Amelia Hired By Staffmark Group to Bring Conve...,"July 29, 2021 • 2 minute read",,Staffmark Group engages Amelia to deliver inte...,https://amelia.ai/press-release/amelia-hired-b...,"{'neg': 0.004, 'neu': 0.87, 'pos': 0.126, 'com...",0.9962,POS,5
2,OrangeShine Hires Amelia to Automate IT Operat...,"May 28, 2021 • 2 minute read",,The leading B2B online fashion wholesale marke...,https://amelia.ai/press-release/orangeshine-hi...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp...",0.9962,POS,4
3,Amelia Recognized as a Leader in Intelligent V...,"May 10, 2021 • 2 minute read",,New report on Hyper-Intelligent Automation hig...,https://amelia.ai/press-release/amelia-recogni...,"{'neg': 0.003, 'neu': 0.898, 'pos': 0.099, 'co...",0.9958,POS,9
4,Kenneth S. Nugent Law Firm Forms Subcompany 1L...,"April 26, 2021 • 2 minute read",,1LAW provides Amelia as a platform that law fi...,https://amelia.ai/press-release/kenneth-s-nuge...,"{'neg': 0.015, 'neu': 0.857, 'pos': 0.128, 'co...",0.9965,POS,4


In [25]:
df.to_excel('amelia_final.xlsx', index=False)